In [1]:
import mujoco
import mediapy as media
import mujoco.viewer as viewer

libEGL warning: failed to open /dev/dri/renderD128: Permission denied

libEGL warning: failed to open /dev/dri/renderD128: Permission denied

libEGL warning: NEEDS EXTENSION: falling back to kms_swrast


In [6]:
model = mujoco.MjModel.from_xml_path('../build/mjpc/tasks/h1/walk/task.xml')
data = mujoco.MjData(model)
print("Model info:")
print(" nq", model.nq)
print(" nv", model.nv)
print(" nu", model.nu)
print(" na", model.na)
print(" njoint", model.njnt)

print("\nModel joints:")
for i in range(model.njnt):
    print("",model.joint(i).name, model.joint(i).type, model.joint(i).qposadr, model.joint(i).dofadr, model.joint(i).qpos0, model.joint(i).range)
    
print("\nModel bodies:")
for i in range(model.nbody):
    print("",model.body(i).name, i, model.body(i).pos)
    
print("\nModel sensors:")
for i in range(model.nsensor):
    print("",model.sensor(i).name, i, model.sensor(i).type, data.sensor(i).data)
    

Model info:
 nq 26
 nv 25
 nu 19
 na 0
 njoint 20

Model joints:
  [0] [0] [0] [0.   0.   1.06 1.   0.   0.   0.  ] [0. 0.]
 left_hip_yaw [3] [7] [6] [0.] [-0.43  0.43]
 left_hip_roll [3] [8] [7] [0.] [-0.43  0.43]
 left_hip_pitch [3] [9] [8] [0.] [-1.57  1.57]
 left_knee [3] [10] [9] [0.] [-0.26  2.05]
 left_ankle [3] [11] [10] [0.] [-0.87  0.52]
 right_hip_yaw [3] [12] [11] [0.] [-0.43  0.43]
 right_hip_roll [3] [13] [12] [0.] [-0.43  0.43]
 right_hip_pitch [3] [14] [13] [0.] [-1.57  1.57]
 right_knee [3] [15] [14] [0.] [-0.26  2.05]
 right_ankle [3] [16] [15] [0.] [-0.87  0.52]
 torso [3] [17] [16] [0.] [-2.35  2.35]
 left_shoulder_pitch [3] [18] [17] [0.] [-2.87  2.87]
 left_shoulder_roll [3] [19] [18] [0.] [-0.34  3.11]
 left_shoulder_yaw [3] [20] [19] [0.] [-1.3   4.45]
 left_elbow [3] [21] [20] [0.] [-1.25  2.61]
 right_shoulder_pitch [3] [22] [21] [0.] [-2.87  2.87]
 right_shoulder_roll [3] [23] [22] [0.] [-3.11  0.34]
 right_shoulder_yaw [3] [24] [23] [0.] [-4.45  1.3 ]
 right

In [7]:
data.body('right_ankle_link')

<_MjDataBodyViews
  cacc: array([0., 0., 0., 0., 0., 0.])
  cfrc_ext: array([0., 0., 0., 0., 0., 0.])
  cfrc_int: array([0., 0., 0., 0., 0., 0.])
  cinert: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
  crb: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
  cvel: array([0., 0., 0., 0., 0., 0.])
  id: 12
  name: 'right_ankle_link'
  subtree_angmom: array([0., 0., 0.])
  subtree_com: array([0., 0., 0.])
  subtree_linvel: array([0., 0., 0.])
  xfrc_applied: array([0., 0., 0., 0., 0., 0.])
  ximat: array([0., 0., 0., 0., 0., 0., 0., 0., 0.])
  xipos: array([0., 0., 0.])
  xmat: array([0., 0., 0., 0., 0., 0., 0., 0., 0.])
  xpos: array([0., 0., 0.])
  xquat: array([0., 0., 0., 0.])
>

In [23]:
model.opt.timestep = 0.03

In [24]:
geom = mujoco.MjvGeom()
start_pos = [0, 0, 0]
end_pos = [1, 1, 1]
width = 0.1
type = mujoco.mjtGeom.mjGEOM_LINE


In [25]:
def add_fw_cost(manager):
    line = manager.add_line()
    line.start = lambda: data.site('upper_torso').xpos
    line.end = lambda: data.site('upper_torso').xpos + data.sensor('torso_forward').data
    line = manager.add_line()
    line.start = lambda: data.body('pelvis').xpos
    line.end = lambda: data.body('pelvis').xpos + data.sensor('pelvis_forward').data
    line = manager.add_line()
    line.start = lambda: data.body('right_ankle_link').xpos
    line.end = lambda: data.body('right_ankle_link').xpos + data.sensor('foot_right_forward').data
    line = manager.add_line()
    line.start = lambda: data.body('left_ankle_link').xpos
    line.end = lambda: data.body('left_ankle_link').xpos + data.sensor('foot_left_forward').data

In [26]:
import time
import mujoco
import mujoco.viewer
from mjv_utils import Line, GeomManager

with mujoco.viewer.launch_passive(model, data) as viewer:
  start = time.time()
  while not viewer.is_running():
    pass
  viewer.opt.flags[mujoco.mjtVisFlag.mjVIS_TRANSPARENT] = 1
  manager = GeomManager(viewer.user_scn)
  add_fw_cost(manager)
  while viewer.is_running():
    step_start = time.time()

    # mj_step can be replaced with code that also evaluates
    # a policy and applies a control signal before stepping the physics.
    mujoco.mj_step(model, data)
    manager.update()
    
    # Example modification of a viewer option: toggle contact points every two seconds.
    # with viewer.lock():
    #   viewer.opt.flags[mujoco.mjtVisFlag.mjVIS_CONTACTPOINT] = int(data.time % 2)
    
    # Pick up changes to the physics state, apply perturbations, update options from GUI.
    viewer.sync()

    # Rudimentary time keeping, will drift relative to wall clock.
    time_until_next_step = model.opt.timestep - (time.time() - step_start)
    
    if time_until_next_step > 0:
      time.sleep(time_until_next_step)
    else:
      print("Simulation not running in real time!")

: 

In [27]:
model = mujoco.MjModel.from_xml_path('./build/mjpc/tasks/humanoid/walk/task.xml')
data = mujoco.MjData(model)
viewer.launch(model=model, data=data)

In [30]:
model = mujoco.MjModel.from_xml_path('./mjpc/tasks/h1/walk/task.xml')
data = mujoco.MjData(model)
viewer.launch(model=model, data=data)